In [1]:
import os 
import re
import torch 
import librosa 
import scipy.signal as sig 
import soundfile as sf
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, WhisperProcessor, WhisperForConditionalGeneration

In [2]:
# wv2_base_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
# wv2_base_proc = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# wv2_large_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")
# wv2_large_proc = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
whisper_large_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3-turbo")
whisper_large_proc = WhisperProcessor.from_pretrained("openai/whisper-large-v3-turbo")

In [31]:
chip_wav = '/home/cogsci-lasrlab/Documents/FANN/K1_wav_kid/K1AM166participant_chip.wav'
ship_wav = '/home/cogsci-lasrlab/Documents/FANN/K1_wav_kid/K1AM166participant_ship.wav'
hotwords = ['chip', 'ship', 'house', 'towel']

### Extracting Whisper Probability 

In [32]:
whisper_vocab_dict = whisper_large_proc.tokenizer.get_vocab()
sorted_whisper_vocab_dict = {k.replace('Ġ', ' '): v for k, v in sorted(whisper_vocab_dict.items(), key = lambda item: item[1])}

In [33]:
count = 0 
new_dict = {}
for k, v in sorted_whisper_vocab_dict.items():
    for h in hotwords: 
        if h in k or h.upper() in k.upper() or h.lower() in k.lower():
            count += 1 
            new_dict[k] = v 
            
        if len(k) < 2 or '<|' in k: 
            count += 1 
            new_dict[k] = v
            
print(count)

ind_list = [v for v in new_dict.values()]

7524


In [42]:
new_dict

{'!': 0,
 '"': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '<': 27,
 '=': 28,
 '>': 29,
 '?': 30,
 '@': 31,
 'A': 32,
 'B': 33,
 'C': 34,
 'D': 35,
 'E': 36,
 'F': 37,
 'G': 38,
 'H': 39,
 'I': 40,
 'J': 41,
 'K': 42,
 'L': 43,
 'M': 44,
 'N': 45,
 'O': 46,
 'P': 47,
 'Q': 48,
 'R': 49,
 'S': 50,
 'T': 51,
 'U': 52,
 'V': 53,
 'W': 54,
 'X': 55,
 'Y': 56,
 'Z': 57,
 '[': 58,
 '\\': 59,
 ']': 60,
 '^': 61,
 '_': 62,
 '`': 63,
 'a': 64,
 'b': 65,
 'c': 66,
 'd': 67,
 'e': 68,
 'f': 69,
 'g': 70,
 'h': 71,
 'i': 72,
 'j': 73,
 'k': 74,
 'l': 75,
 'm': 76,
 'n': 77,
 'o': 78,
 'p': 79,
 'q': 80,
 'r': 81,
 's': 82,
 't': 83,
 'u': 84,
 'v': 85,
 'w': 86,
 'x': 87,
 'y': 88,
 'z': 89,
 '{': 90,
 '|': 91,
 '}': 92,
 '~': 93,
 '¡': 94,
 '¢': 95,
 '£': 96,
 '¤': 97,
 '¥': 98,
 '¦': 99,
 '§': 100

In [43]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(list(new_dict.keys()), kenlm_model_path=None) 

Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


In [44]:
def clean(text):
    text = re.sub(r'\<\|[^)]*?\|\>', '', text).strip()
    text = re.sub(r'-.!', '', text).strip()
    text = re.sub(r"[^\w\s]", "", text).strip()
    text = text.replace('(', '')
    text = text.replace('\n', '')
    text = text.replace(')', '')
    text = text.replace('?', '')
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text

In [45]:
audio_input, _ = librosa.load(chip_wav, sr=16000)
input_values = whisper_large_proc(
    audio_input, 
    sampling_rate=16000, 
    return_tensors="pt"
).input_features

with torch.no_grad():
    predicted_ids = whisper_large_model.generate(
        input_values, 
        return_dict_in_generate=True, 
        output_scores=True
    )
    
comp = torch.stack(list(predicted_ids.scores), dim=0).squeeze()
logits = torch.exp(comp).cpu().detach().numpy()

reduced_logits = logits[:, ind_list]

transcription = decoder.decode(
    reduced_logits, 
    hotwords=hotwords,
    hotword_weight=100.0
)

text = clean(transcription).strip()
print(text)

chip


In [46]:
new_tries = decoder.decode_beams(
    reduced_logits, 
    hotword_weight=100.0,
    beam_width=100,
    token_min_logp=-100000,
    beam_prune_logp=-100000
)

In [47]:
h_prob_dict = {}
for h in hotwords:
    h_prob_dict[h] = 0
    
for k in new_tries:
    if clean(k[0]) in hotwords:
        h_prob_dict[clean(k[0])] += np.exp(k[3])
        
print(h_prob_dict)

{'chip': 1.00000000000004, 'ship': 0, 'house': 0, 'towel': 0}


### Extracting Wav2Vec2 Probability 

In [14]:
wv2_vocab_dict = wv2_base_proc.tokenizer.get_vocab()
sorted_wv2_vocab_dict = {k: v for k, v in sorted(wv2_vocab_dict.items(), key = lambda item: item[1])}

In [15]:
count = 0 
new_dict = {}
for k, v in sorted_wv2_vocab_dict.items():
    for h in hotwords: 
        if h in k or h.upper() in k.upper() or h.lower() in k.lower():
            count += 1 
            new_dict[k] = v 
            
        if len(k) < 2 or '<|' in k: 
            count += 1 
            new_dict[k] = v
            
print(count)

ind_list = [v for v in new_dict.values()]

112


In [16]:
decoder = build_ctcdecoder(list(new_dict.keys()))

In [17]:
audio_input, _ = librosa.load(chip_wav, sr=16000)
input_values = wv2_large_proc(
    audio_input, 
    sampling_rate=16000, 
    return_tensors="pt"
).input_values

with torch.no_grad():
    predicted_ids = wv2_large_model.generate(
        input_values, 
        return_dict_in_generate=True, 
        output_scores=True
    )
    
comp = torch.stack(list(predicted_ids.scores), dim=0).squeeze()
logits = torch.exp(comp).cpu().detach().numpy()

reduced_logits = logits[:, ind_list]

transcription = decoder.decode(
    reduced_logits, 
    hotwords=hotwords,
    hotword_weight=100.0
)

text = clean(transcription).strip()
print(text)

TypeError: The current model class (Wav2Vec2ForCTC) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].